Darren Colby

MSCA 32020

Value Iteration

The purpose of this notebook is to get an intuiton for how value iteration works by implementing it from scratch and use the implementation to solve a simple grid problem. Towards that purpose, I have tried to make this implementation as clear as possible by using dictionaries instead of matrices, using action names like "Right" instead of an arbitrary number or letter, and labeling variables with names like transition_probabilities instead of T or P.

In [1]:
using Revise
using StatsBase
using LinearAlgebra

## Step 1: Define an MDP Struct

To make our lives easier we will make a struct to store all the components for a simple MDP. We can initialize an MDP by passing a tuple of states, a tuple of actions, a dictionary of rewards, and a dictionary of transition probabilities or dictionary of dictionaries of transition probabilities if we want our transition probabilities to depend on the action the agent takes.

In [2]:
"""Struct to store the necessary componeents of a Markov Decision Process"""
struct MDP
    states::Tuple
    actions::Tuple
    rewards::Dict
    transition_probabilities::Dict
    num_states::Int64
    num_actions::Int64
    γ::Float64
    values::Dict
    π::Vector{Float64}
end


"""
    MDP(states, actions, rewards, transitiion_probabilities)

    Create an MDP struct to solve
"""
function MDP(states::Tuple, actions::Tuple, rewards::Dict, transition_probabilities::Dict, 
    γ::Float64=0.9)

    # Initialize values to zero    
    values = Dict(state => 0.0 for state in states)

        MDP(states, actions, rewards, transition_probabilities, size(states, 1), 
            size(actions, 1), γ, values, zeros(size(states, 1)))
end

MDP

## Step 2: Update Values for Each Action

To solve our MDP with value iteration we will define functions inside out. In other words, we will start by defining a function to update the values for a single action using the Bellman Optimality Condition. Then, we can call this functiion for each action and update its value. To do this we will get our next action (s') by using our current state and action to sample from the transition probabilities. Then, we will update the value of our action by adding its immediate reward r(s, a) from the rewards dictionary to the discounted sum of the product of our transition probabilities P(s'|s, a) multiplied by each possible next state value (s') in our values dictionary. This update will be of the form $V(a) = r(s, a) + \gamma{\Sigma{P(s'|s, a)*V(a')}}$ 

In [3]:
"""
gettransitionprobability(mdp, state, action)

Get the next state and its transition probability
"""
function gettransitionprobability(mdp::MDP, state, action)
    # Get the next state based on its probability given the current state and action
    possible_states = collect(keys(mdp.transition_probabilities[action][state]))
    state_probs = ProbabilityWeights(collect(values(mdp.transition_probabilities[action][state])))
    next_state = sample(possible_states, state_probs)

    return mdp.transition_probabilities[action][state][next_state], next_state
end

"""
bellmanupdate(mdp, action_values, state, action)

Update the estimated value of an action at a given state using the Bellman Optimality Condition
"""
function bellmanupdate(mdp::MDP, action_values::Dict, state, action)

    # Get transition probabilities from the given state and action
    transition_probabilities = values(mdp.transition_probabilities[action][state])

    # Get the immediate reward for that state
    reward = mdp.rewards[state][action]

    # Immediate reward plus discounted expectation of values for next possible states
    action_values[action] = float(reward) + (mdp.γ*dot(transition_probabilities, values(mdp.values)))
    
    return action_values
end

bellmanupdate

## Step 3: Estimate the Values for Each State

The value for a state under an optimal policy is $r(s, a) + \gamma max_a{\Sigma{P(s'|s, a)*V(a')}}.$  This is what the function above does, only, it does not do the max operation. To estimate the value of our states we iterate through each state, iterate through each action using the function above, and assign the value of a state as the value of the action with the highest expected discounted reward that the agent can take.

In [4]:
"""
estimateV!(mdp)

Estimate the value of a state
"""
function estimateV!(mdp::MDP)
    Δ = Vector{Float64}(undef, mdp.num_states)
    for (i, state) in enumerate(mdp.states)
        temp = deepcopy(mdp.values[state]) 
        action_values = Dict(action => 0.0 for action in mdp.actions)
        
        # Collect the rewards for taking each action at each state
        for action in mdp.actions

            # If the agent hits the absorbing state
            if gettransitionprobability(mdp, state, action) ∈ (0, 1)
                mdp.values[state] = 0
                break
            else
                # Get updated values for each action
            action_values = bellmanupdate(mdp, action_values, state, action)
            end

        # Update the MDP value with the highest temp value caused by the best action
        mdp.values[state] = maximum(collect(values(action_values)))
        Δ[i] = abs(temp - mdp.values[state])
        end
    end
    return maximum(Δ)
end

estimateV!

## Step 4: Finding the Optimal Policy

Now that we have a way to estimate the optimal values for each state, we can use them to find the optimal policy, give by $\pi^* = argmax_a\Sigma_{s'}{P(s'|s, a)v(s')}.$ To do this we use nearly the same method we used to estimate the value function. We iterate through each state and each action, substitute our estimated state values into the equation above, use it to find the best action at each state, and map those states and actions.

In [5]:
"""
getpolicy(mdp)

Estimate the optimal policy for an MDP using estimated state values
"""
function getpolicy(mdp::MDP)
    policy = Dict()
    for state in mdp.states in 
        action_values = Dict()
        
        for action in mdp.actions
           action_values = bellmanupdate(mdp, action_values, state, action) 
        end
        policy[state] = findmax(action_values)[2]
    end
    return policy
end

getpolicy

## Step 5: Putting It All Together

To solve an MDP is estimate V for many iterations until the values stop changing a lot. Then, we can use the function defined above to coninually improve the estimated state values within a certain tolerance. Using these estimated state values we use the getpolicyfunction to find an optimal policy.

In [6]:
"""
solve!(mdp, tolerance)

Solve a Markov Decision Process using value iteration
"""
function solve!(mdp::MDP, tolerance::Float64=1e-5)
    iteration = 0
    Δ = estimateV!(mdp)

    while Δ > tolerance
        iteration += 1
        Δ = estimateV!(mdp)
        println("Change in state value: " * string(Δ))

        if Δ < tolerance
            break
        end
    end
    return getpolicy(mdp), iteration
end

solve!

## A Simple MDP Problem

To show that the above implementation works, we can solve a very simple problem. Similar to the robot problem we have seen in class, we can imagine a segment of grid sqaures (states) 0, 1, 2, and 3. The goal of the robot is to get to square 3, which has a reward of 5. The other squares impose a fuel cost of -1. Accordingly, the robot's action space consists of the actions "Right" and "Left." Below is also a dictionary of transition probabilities that defines the probability of starting in one state and ending in another. Like we saw in class, there can be different transition matrices for each action, as there is for the example here. One thing to notice is that this is an infinite horizon problem--a new episode does not restart when the robot reaches square 3--instead, square 3 is an absorbing state and any action the robot takes in this absorbing state has a reward of zero. Square 0 is a bit different because square 0 is not the robots goal, so if the robot stays in square 0 it can still go right towards its goal. It would also be possible to solve the full robot problem from class but that would require manually typing more states, actions, and values, which I'm too lazy to do.

In [7]:
states = Tuple(i for i in 0:3)
actions = tuple("Left", "Right")
rewards = Dict(0 => Dict("Left" => -1, "Right" => -1), 
           1 => Dict("Left" => -1, "Right" => -1), 
           2 => Dict("Left" => -1, "Right" => 5),
           3 => Dict("Left" => 0, "Right" => 0))
transition_probabilities = Dict("Left" => Dict(0 => Dict(0 => 0.8, 1 => 0.2, 2 => 0, 3 => 0), 
                                               1 => Dict(0 => 0.8, 1 => 0, 2 => 0.2, 3 => 0), 
                                               2 => Dict(0 => 0, 1 => 0.8, 2 => 0, 3 => 0.2), 
                                               3 => Dict(0 => 0, 1 => 0, 2 => 0, 3 => 1)), 
                                "Right" => Dict(0 => Dict(0 => 0.2, 1 => 0.8, 2 => 0, 3 => 0), 
                                                1 => Dict(0 => 0.2, 1 => 0, 2 => 0.8, 3 => 0), 
                                                2 => Dict(0 => 0, 1 => 0.2, 2 => 0, 3 => 0.8), 
                                                3 => Dict(0 => 0, 1 => 0, 2 => 0, 3 => 1)))

Dict{String, Dict{Int64, Dict{Int64}}} with 2 entries:
  "Right" => Dict(0=>Dict{Int64, Real}(0=>0.2, 2=>0, 3=>0, 1=>0.8), 2=>Dict{Int…
  "Left"  => Dict(0=>Dict{Int64, Real}(0=>0.8, 2=>0, 3=>0, 1=>0.2), 2=>Dict{Int…

In [9]:
simple_robot_mdp = MDP(states, actions, rewards, transition_probabilities)
solve!(simple_robot_mdp)

Change in state value: 3.294144
Change in state value: 2.3717836800000005
Change in state value: 0.6147663298560007
Change in state value: 0.23002953889097233
Change in state value: 0.0825459302565994
Change in state value: 0.02871165260908315
Change in state value: 0.009944082240489927
Change in state value: 0.0034420075516421456
Change in state value: 0.0011913039277553494
Change in state value: 0.0004123141420593335
Change in state value: 0.00014270302312979766
Change in state value: 4.93898851234853e-5
Change in state value: 1.7093966337089483e-5
Change in state value: 5.9162657106703875e-6


(Dict{Any, Any}(0 => "Right", 2 => "Right", 3 => "Right", 1 => "Right"), 14)